In [17]:
!pip install -q pyomo

In [18]:
from pyomo.environ import * 

In [19]:
import numpy as np


[R] Write a general optimization problem using the actual objective provided in the description. You may assume that b0 ≥ 0 and b1, b2 do not have bounds on them. Is the optimization
problem linear?

Minimize:   ∑ e [ i ]

and the additional constraints are :

ei ≥ Ri=Pi−(bo+b1Li+b2Ei)

ei≥ −Ri=−Pi+(bo+b1Li+b2Ei)

 for i=1,2,...,6



APPROACH 1

In [20]:
coef = np.loadtxt('lab6_ex3_coeff.txt', delimiter=',')

In [21]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])

(13, 10)
Num rows: 13
Num cols: 10


In [22]:
model = ConcreteModel()

In [23]:
M = coef.shape[0]-1
N = coef.shape[1]-1
print(M,N)

12 9


In [24]:
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(9,)
[0. 0. 0. 1. 1. 1. 1. 1. 1.]


In [25]:
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(12, 9)
[[-1.0e+00 -1.8e+03 -2.5e+02 -1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00]
 [-1.0e+00 -2.8e+03 -4.0e+02  0.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00]
 [-1.0e+00 -3.5e+03 -4.5e+02  0.0e+00  0.0e+00 -1.0e+00  0.0e+00  0.0e+00
   0.0e+00]
 [-1.0e+00 -8.0e+02 -1.0e+02  0.0e+00  0.0e+00  0.0e+00 -1.0e+00  0.0e+00
   0.0e+00]
 [-1.0e+00 -7.0e+02 -2.0e+02  0.0e+00  0.0e+00  0.0e+00  0.0e+00 -1.0e+00
   0.0e+00]
 [-1.0e+00 -5.0e+02 -5.0e+01  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
  -1.0e+00]
 [ 1.0e+00  1.8e+03  2.5e+02 -1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00]
 [ 1.0e+00  2.8e+03  4.0e+02  0.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00]
 [ 1.0e+00  3.5e+03  4.5e+02  0.0e+00  0.0e+00 -1.0e+00  0.0e+00  0.0e+00
   0.0e+00]
 [ 1.0e+00  8.0e+02  1.0e+02  0.0e+00  0.0e+00  0.0e+00 -1.0e+00  0.0e+00
   0.0e+00]
 [ 1.0e+00  7.0e+02  2.0e+02  0.0e+00  0.0e+00  0.0e+00  0.0e+00 -1.0e+00
   0.0e+00]
 [ 1.0e+00  5.0e+02  5.0e+01  0.0e+00  0.0e+00

In [26]:
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(12,)
[-175000. -150000. -125000.  -90000.  -80000. -120000.  175000.  150000.
  125000.   90000.   80000.  120000.]


In [27]:
col_indices_b = np.arange(3)
print(col_indices_b)
model.b = Var(col_indices_b,domain=Reals)

[0 1 2]


In [28]:
col_indices_e = np.arange(6)
print(col_indices_e)
model.e = Var(col_indices_e,domain=NonNegativeReals)

[0 1 2 3 4 5]


In [29]:
row_indices = np.arange(M)
print(row_indices)

[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [30]:
model.constraints = ConstraintList()

In [31]:
for i in row_indices:
  model.constraints.add(summation(constr_coef[i][0:3],model.b) + summation(constr_coef[i][3:N],model.e)  <= constr_rhs[i])

In [32]:

model.objective = Objective(expr = summation(obj_coef[3:N],model.e), sense=minimize)

In [33]:
model.pprint()

3 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    e_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

2 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
    e : Size=6, Index=e_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : Fa

In [34]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [35]:
opt_cbc = SolverFactory('cbc')

In [36]:
result = opt_cbc.solve(model)
print(result)


Problem: 
- Name: unknown
  Lower bound: 125454.5455
  Upper bound: 125454.5455
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 10
  Number of nonzeros: 6
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 8
  Error rc: 0
  Time: 0.02327585220336914
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [37]:
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in range(3):
  print('b[',i,'] = ', model.b[i].value)
for j in range(6):
  print('e[',j,'] = ', model.e[j].value)


Solver status: ok
Solver termination condition: optimal

Objective =  125454.54500000001

Decision Variables
b[ 0 ] =  104181.82
b[ 1 ] =  67.272727
b[ 2 ] =  -356.36364
e[ 0 ] =  38818.182
e[ 1 ] =  0.0
e[ 2 ] =  54272.727
e[ 3 ] =  32363.636
e[ 4 ] =  0.0
e[ 5 ] =  1.671118e-11


APPROACH 2:
 assume that \
$|R_{i}| = x_{i}+y_{i}$ \
and
$R_{i} = x_{i}-y_{i} \ ,\ $ \
 objective function is : \
$ Minimum: \sum_{i} (x_{i}+y_{i}) \ $  \
 constraints are: \
$P_{i}-(b_{0}+b_{1}L_{i}+b_{2}E_{i})-x_{i}+y_{i} = 0 ,$ \
$P_{i}-(b_{0}+b_{1}L_{i}+b_{2}E_{i})-x_{i}-y_{i} <= 0 ,$ \
$-P_{i}+(b_{0}+b_{1}L_{i}+b_{2}E_{i})-x_{i}-y_{i} <= 0 ,$ \




In [39]:
coef = np.loadtxt('lab6_ex3_coeff2.txt', delimiter=',')

In [40]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])

(25, 16)
Num rows: 25
Num cols: 16


In [41]:
model = ConcreteModel()

In [42]:
M = coef.shape[0]-1
N = coef.shape[1]-1
print(M,N)

24 15


In [43]:
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(15,)
[0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [45]:
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(24, 15)
[[ 1.0e+00  1.8e+03  2.5e+02  1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 1.0e+00  2.8e+03  4.0e+02  0.0e+00  1.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 1.0e+00  3.5e+03  4.5e+02  0.0e+00  0.0e+00  1.0e+00  0.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 1.0e+00  8.0e+02  1.0e+02  0.0e+00  0.0e+00  0.0e+00  1.0e+00  0.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00 -1.0e+00  0.0e+00  0.0e+00]
 [ 1.0e+00  7.0e+02  2.0e+02  0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00
   0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00 -1.0e+00  0.0e+00]
 [ 1.0e+00  5.0e+02  5.0e+01  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00 -1.0e+00]
 [-1.0e+00 -1.8e+03 -2.5e+02 -1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00
   0.0e+00  1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [-1.0e+00 

In [46]:
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(24,)
[ 175000.  150000.  125000.   90000.   80000.  120000. -175000. -150000.
 -125000.  -90000.  -80000. -120000. -175000. -150000. -125000.  -90000.
  -80000. -120000.  175000.  150000.  125000.   90000.   80000.  120000.]


In [47]:
col_indices_b = np.arange(3)
print(col_indices_b)
model.b = Var(col_indices_b,domain=Reals)

[0 1 2]


In [48]:
col_indices_x = np.arange(6)
print(col_indices_x)
model.x = Var(col_indices_x,domain=NonNegativeReals)

[0 1 2 3 4 5]


In [49]:
col_indices_y= np.arange(6)
print(col_indices_y)
model.y = Var(col_indices_y,domain=NonNegativeReals)

[0 1 2 3 4 5]


In [50]:
row_indices = np.arange(M)
print(row_indices)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [51]:
model.constraints = ConstraintList()

In [52]:
for i in row_indices:
  model.constraints.add(summation(constr_coef[i][0:3],model.b) + summation(constr_coef[i][3:9],model.x) + summation(constr_coef[i][9:15],model.y)  <= constr_rhs[i])

In [53]:
model.objective = Objective(expr = summation(obj_coef[3:9],model.x) + summation(obj_coef[9:15],model.y), sense=minimize)

In [54]:
model.pprint()

4 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

3 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
 

In [55]:
!apt-get install -y -qq coinor-cbc

In [56]:
opt_cbc = SolverFactory('cbc')

In [58]:
result = opt_cbc.solve(model)
print(result)


Problem: 
- Name: unknown
  Lower bound: 125454.5455
  Upper bound: 125454.5455
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 16
  Number of nonzeros: 12
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 9
  Error rc: 0
  Time: 0.02042675018310547
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [61]:
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in range(3):
  print('b[',i,'] = ', model.b[i].value)
for j in range(6):
  print('x[',j,'] = ', model.x[j].value)
  print('y[',j,'] = ', model.y[j].value)

Solver status: ok
Solver termination condition: optimal

Objective =  125454.545

Decision Variables
b[ 0 ] =  104181.82
b[ 1 ] =  67.272727
b[ 2 ] =  -356.36364
x[ 0 ] =  38818.182
y[ 0 ] =  0.0
x[ 1 ] =  0.0
y[ 1 ] =  0.0
x[ 2 ] =  0.0
y[ 2 ] =  54272.727
x[ 3 ] =  0.0
y[ 3 ] =  32363.636
x[ 4 ] =  0.0
y[ 4 ] =  0.0
x[ 5 ] =  0.0
y[ 5 ] =  0.0
